# Desafío 2: entrenamiento de embeddings con Gensim sobre Ética a Nicómaco

## Obtención y análisis del dataset

In [1]:
!pip install numpy gensim

In [2]:
# se importan librerías esenciales
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import multiprocessing
from gensim.models import Word2Vec


In [4]:
# se lee el corpus de Ética a Nicómaco
with open('etica_nicomaco.txt', 'r', encoding='utf-8') as f:
    texto = f.read()
# se parte el texto en lineas para simular documentos
lineas = texto.split('\n')
df = pd.DataFrame(lineas, columns=['texto'])
df = df[df['texto'].str.strip() != '']
df.reset_index(drop=True, inplace=True)
df

,texto
0,"Cualquier arte y cualquier doctrina, y asimism..."
1,Pero como sean muchas las acciones y las artes...
2,"En todas cuantas hay de esta suerte, que debaj..."
3,"Presupuesta esta verdad en el capítulo pasado,..."
4,Demuestra asimismo cómo el considerar este fin...
...,...
1062,Después que con muy claras razones ha probado ...
1063,Después désta es la más perfeta la que es conf...
1064,Capítulo 9. Del saber y de la práctica en esta...
1065,Trata el filósofo en este capítulo de la neces...


In [5]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 1067


## Preprocesamiento

In [6]:
from tensorflow.keras.preprocessing.text import text_to_word_sequence
sentence_tokens = []
for _, row in df.iterrows():
    sentence_tokens.append(text_to_word_sequence(row['texto']))
sentence_tokens[:3]

[['cualquier',
  'arte',
  'y',
  'cualquier',
  'doctrina',
  'y',
  'asimismo',
  'toda',
  'acción',
  'y',
  'elección',
  'parece',
  'que',
  'a',
  'algún',
  'bien',
  'es',
  'enderezada',
  'por',
  'tanto',
  'discretamente',
  'difinieron',
  'el',
  'bien',
  'los',
  'que',
  'dijeron',
  'ser',
  'aquello',
  'a',
  'lo',
  'cual',
  'todas',
  'las',
  'cosas',
  'se',
  'enderezan',
  'pero',
  'parece',
  'que',
  'hay',
  'en',
  'los',
  'fines',
  'alguna',
  'diferencia',
  'porque',
  'unos',
  'de',
  'ellos',
  'son',
  'acciones',
  'y',
  'otros',
  'fuera',
  'de',
  'las',
  'acciones',
  'son',
  'algunas',
  'obras',
  'y',
  'donde',
  'los',
  'fines',
  'son',
  'algunas',
  'cosas',
  'fuera',
  'de',
  'las',
  'acciones',
  'allí',
  'mejores',
  'son',
  'las',
  'obras',
  'que',
  'las',
  'mismas',
  'acciones'],
 ['pero',
  'como',
  'sean',
  'muchas',
  'las',
  'acciones',
  'y',
  'las',
  'artes',
  'y',
  'las',
  'ciencias',
  'de',
  'n

## Vectorización

In [13]:
from gensim.models.callbacks import CallbackAny2Vec
class callback(CallbackAny2Vec):
    def __init__(self):
        self.epoch = 0
    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch % 100 == 0:
          if self.epoch == 0:
              print('Loss after epoch {}: {}'.format(self.epoch, loss))
          else:
              print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [14]:
w2v_model = Word2Vec(min_count=3, window=2, vector_size=400, negative=10, workers=3, sg=1)

In [15]:
w2v_model.build_vocab(sentence_tokens)

In [16]:
print("Cantidad de documentos en el corpus:", w2v_model.corpus_count)

Cantidad de documentos en el corpus: 1067


In [17]:
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 2351


In [18]:
w2v_model.train(sentence_tokens, total_examples=w2v_model.corpus_count, epochs=1000, compute_loss=True, callbacks=[callback()])

Loss after epoch 0: 279989.40625
Loss after epoch 100: 125263.0
Loss after epoch 200: 83530.0
Loss after epoch 300: 69320.0
Loss after epoch 400: 70724.0
Loss after epoch 500: 89636.0
Loss after epoch 600: 89408.0
Loss after epoch 700: 4272.0
Loss after epoch 800: 3456.0
Loss after epoch 900: 3064.0


(64364079, 108188000)

In [19]:
w2v_model.save('embeddings_nicomaco.model')
print('Modelo guardado correctamente.')

Modelo guardado correctamente.


## Ensayos

In [20]:
from gensim.models import Word2Vec
embeddings_model = Word2Vec.load('embeddings_nicomaco.model')

### Palabras más similares

In [21]:
# Ejemplo con palabras clave del texto de Aristóteles
embeddings_model.wv.most_similar(positive=["virtud"], topn=10)

[('tantos', 0.24553069472312927),
 ('¿habemos', 0.2378721982240677),
 ('civiles', 0.23523928225040436),
 ('cuarto', 0.23223985731601715),
 ('cítara', 0.22918903827667236),
 ('perfección', 0.22853094339370728),
 ('cuanta', 0.22577226161956787),
 ('heroica', 0.22413939237594604),
 ('encaminada', 0.2191847860813141),
 ('imperfetas', 0.21706369519233704)]

In [22]:
embeddings_model.wv.most_similar(positive=["felicidad"], topn=10)

[('mocedad', 0.25661519169807434),
 ('tomamos', 0.2502357065677643),
 ('religión', 0.24681293964385986),
 ('inquirimos', 0.24583202600479126),
 ('sumo', 0.24451711773872375),
 ('perfección', 0.24200183153152466),
 ('perfecta', 0.241202712059021),
 ('contemplación', 0.23899519443511963),
 ('sacrificios', 0.23155485093593597),
 ('atribuir', 0.22866062819957733)]

In [23]:
embeddings_model.wv.most_similar(positive=["justicia"], topn=10)

[('sinjusticia', 0.28551408648490906),
 ('agravios', 0.2611428201198578),
 ('particular', 0.25084707140922546),
 ('injusto', 0.24848788976669312),
 ('prudencia', 0.23119963705539703),
 ('justo', 0.22457338869571686),
 ('agravio', 0.22165872156620026),
 ('enoja', 0.21897433698177338),
 ('quisieron', 0.21815486252307892),
 ('carece', 0.2172369509935379)]

### Palabras menos similares

In [24]:
embeddings_model.wv.most_similar(negative=["virtud"], topn=10)

[('carecen', 0.078160859644413),
 ('largo', 0.07523007690906525),
 ('hacía', 0.04933968186378479),
 ('miran', 0.04600010812282562),
 ('rústicos', 0.04386681690812111),
 ('mandar', 0.03387957438826561),
 ('bienaventurados', 0.030427519232034683),
 ('romanos', 0.02854960598051548),
 ('perdido', 0.02837335877120495),
 ('siéndolo', 0.02462499588727951)]

### Tests de analogías

In [25]:
# Ejemplo: Emb(virtud):Emb(placer)::Emb(felicidad):Emb(?)
embeddings_model.wv.most_similar(positive=['felicidad', 'placer'], negative=['virtud'], topn=1)

[('pesadas', 0.28799501061439514)]

## Visualizaciones

In [26]:
from sklearn.manifold import TSNE
import numpy as np
def reduce_dimensions(model, num_dimensions=2):
    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)
    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)
    return vectors, labels

In [28]:
import plotly.express as px
vecs, labels = reduce_dimensions(embeddings_model)
MAX_WORDS=200
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show()

In [29]:
vecs, labels = reduce_dimensions(embeddings_model,3)
fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
fig.show()

# Conclusiones

### Lo que funcionó:

El modelo Word2Vec se entrenó bien sobre el texto de Aristóteles y logró capturar relaciones semánticas interesantes. Las palabras más similares a "virtud" incluyeron términos como "perfección" y "heroica", lo cual tiene sentido filosóficamente. Para "felicidad" encontró palabras como "contemplación" y "perfección", que van bien con la idea de eudaimonia.

### Lo que no funcionó:

El corpus era chico y eso limitó la calidad de los embeddings. Las analogías que generó no siempre fueron intuitivas filosóficamente, como cuando relacionó "felicidad" con "pesadas" en una analogía.